# Data Filtering Using AI 🤖

## Index:

* ###  0. Requirements and Structure
* ###  1. Libraries
* ###  2. Google Sheets
* ###  3. Data Filtering by Name
* ###  4. Data Filtering using AI
* ###  5. Google Slides
* ###  6. Problem Solving
* ###  7. Future Improvements

## 0. Requirements and Structure

The following Requirements must be met before executing this script:

- Have a Google Drive account
- Create a Google Cloud account
- Create a project on Google Cloud
- Enable Drive, Sheets and Slides API credentials to the project
- Download API keys JSON file and store it in the same location as this Jupyter Notebook
- Google Sheets files needs to be created or uploaded manually and shared with your ``service account email``. In my case I used: 

  ``sergi-382@roche2023.iam.gserviceaccount.com``

- The Google Slides template file needs to be manually created and shared with the ``service account email``. It must follow the given data structure by the assignment guidelines


## 1. Libraries

In [ ]:
!pip install numpy
!pip install pandas
!pip install openai
!pip install gspread
!pip install ipywidgets
!pip install oauth2client
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [1]:
# Google APIs
from googleapiclient import discovery
from googleapiclient.discovery import build

# Working with Google Sheets and Slides
from google.oauth2 import service_account
import gspread

# Interactive widgets for automatic text input
from ipywidgets import interact, widgets

# Pandas and numpy to work with df
import pandas as pd
import numpy as np

# Open AI API
import openai

The credentials JSON file can be downloaded from the [Google Cloud Platform](https://console.cloud.google.com) on your service account. Given the structure of this script, the credentials file needs to be stored and located in the same location as this Jupyter Notebook

In [2]:
service_account_file = 'roche2023.json' # My personal credentials

In [3]:
# Necessary scopes to access Extract data from Google Sheets
scopes = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']

## 2. Google Sheets

### 2.1 Function

Extracts data from any Google Sheets file. It is not only limited to the initial structure of ``ID, Name, Occupation, Country, Age``, it can extract data with any structure. It is only required that the file data has a **header**.

Args
* filename (str) The name of the Google Sheets file to extract data from
* service_account_file (str) The path to the .JSON file with the service_account credentials
* scopes (list) Necessary scopes to carry out the authentication

Returns

* df (dataframe) Clean data from the Google Sheets 
* header (str) Header row

In [4]:
def extract_data_from_gs(filename, service_account_file, scopes):
    
    try:
        creds = None
        # Loads service account credentials
        creds = service_account.Credentials.from_service_account_file(service_account_file, scopes = scopes)
        
        # Authorizing Google client, opening the file and retrieving the data from it
        client = gspread.authorize(creds)
        google_sheet = client.open(filename).sheet1
        result = google_sheet.get_all_values()
        
        # Cleaning missing and faulty entries in the data
        df = pd.DataFrame.from_records(result[1:], columns=result[0])
        header = str(df.columns.tolist()) # Converting the header to string
        df.replace('', np.nan, inplace=True)
    
        deleted_rows = df[df.isnull().any(axis=1)] # Stores the deleted rows 
        df.dropna(inplace=True)
        
        # Checks all columns with numbers and converts it to int so filtering operations can be conducted at later stages
        df = df.apply(pd.to_numeric, errors='ignore')
        numeric_columns = df.select_dtypes(include='number').columns
        df[numeric_columns] = df[numeric_columns].astype(int)
        
        if len(deleted_rows) > 0:
            print(f"The following rows with missing or null data have been deleted\n {deleted_rows}") # Displays deleted rows
    
        return df, header
    
    except gspread.SpreadsheetNotFound:
        print(f"Error: Could not find '{filename}' file.") # Error given if the filename is not correct
        return None

### 2.2 Google Sheets filename input

In [5]:
# Interactive widget so it is easier to input the data, no need to execute this cell again
filename_widget = widgets.Text(placeholder='Type filename of the Google Slides file containing the employee data',
                                   layout=widgets.Layout(width='500px'))
display(filename_widget)

Text(value='', layout=Layout(width='500px'), placeholder='Type filename of the Google Slides file containing t…

### 2.3 Display of the results

In [6]:
df, header = extract_data_from_gs(filename_widget.value, service_account_file, scopes)
df

The following rows with missing or null data have been deleted
       ID      Name    Occupation  Country  Age
36    37   Lillian           NaN   France   47
166  167       NaN           NaN      NaN  NaN
332  333      Mila           NaN    India   55
406  407       NaN    Accountant      USA   55
846  847    Peyton           NaN  Austria   34
854  855       Ivy           NaN    Spain   24
867  NaN  Serenity  Data Analyst      USA   29
977  978     Rylee        Intern      USA  NaN
990  991      Aria           NaN  Austria   34


,ID,Name,Occupation,Country,Age
0,1,Jon,Data Scientist,USA,35
1,2,Deadpool,Data Analyst,UK,28
2,3,James,Data Engineer,USA,33
3,4,Wolverine,Software Engineer,Canada,18
4,5,Jon,Accountant,Finnland,23
...,...,...,...,...,...
994,995,Paisley,Accountant,Spain,44
995,996,Ivy,Software Engineer,Switzerland,65
996,997,Penelope,Developer,Austria,18
997,998,Aubrey,Accountant,France,59


## 3 Data Filtering by Name

Finds all instances of the given name. Given the nature of this query, this cell must **only** be executed if a Google Sheets file containing the employee data structure has been previously loaded

In [7]:
# Input the emmployee name query and execute the next cell
employee_name_widget = widgets.Text(placeholder='Type employee name')
display(employee_name_widget)

Text(value='', placeholder='Type employee name')

In [8]:
filtered_df = df.loc[df['Name'] == employee_name_widget.value]
filtered_df

,ID,Name,Occupation,Country,Age
46,47,Charlotte,Executive,Austria,50
137,138,Charlotte,Executive,Germany,33
232,233,Charlotte,Developer,Switzerland,27
342,343,Charlotte,Accountant,Austria,25
379,380,Charlotte,Developer,Switzerland,46
413,414,Charlotte,Executive,India,30
498,499,Charlotte,Intern,France,62
503,504,Charlotte,Intern,India,34
605,606,Charlotte,Accountant,Italy,25
655,656,Charlotte,Data Analyst,UK,41


## 4. Data Filtering Using AI

I decided to also implement the capability of filtering the data contained in any Google Sheets. 

The only necessary input is a written explanation of what the user wants to find. 

In [9]:
openai.api_key = 'Your own OPEN API KEY' # This is my personal OpenAI API key

model_engine = 'text-davinci-003' # The most advanced AI model currently offered by the API

### 4.1 Function

Uses AI to find all instances of a written data query. Using the OpenAI API it sends a prompted request following some guidelines so that the API answer follows a certain structure.

Args
* prompt (str) Written user query
* header (str) Header of the df


Returns

* text (str) pandas df query code


In [10]:
def chatgpt_query(prompt, header):
    # What is sent onto the AI model as a request
    initial_prompt= 'I am going to pass a query in order to find all employees who meet certain requirements inside of a pandas dataframe. Translate those requirements in terms of the header which is ' + header + ' \n\nGive the commands in relationship with the structure of the given header. The answer needs to follow the following df queries in the next examples:\nExample1\nQuery: I want all employees whos name is Jon and live in Spain\n\ndf.loc[(df["Name"] == "Jon") & (df["Country"] == "Spain")]\n\nExample 2\nQuery: I want all employees that live in the UK and are Data scientists\ndf.loc[(df["Country"] == "UK") & (df["Occupation"] == "Data scientist")]\n\nYour answer must be precise and short, JUST the code. Write it in plane text so it is easier to read. You are only allowed to use one single line of code. You get bonus points for shorter answer and for the least amount of code lines. You are NOT allowed to write 2 lines of code or you will lose points. If you give the answer in 2 separate lines of code, you will be deducted points from the total score'
    full_prompt = initial_prompt + '\nNow my query is:\n' + prompt + '\n Only provide your answer with the code'
    
    # Tuning of the AI model parameters allowing to further customize the response
    completion = openai.Completion.create(engine=model_engine,
                                          prompt=full_prompt,
                                          temperature=0.6,
                                          max_tokens=1000,
                                          top_p=1,
                                          frequency_penalty=0.2,
                                          presence_penalty=0)
    # Answer from the request
    text = completion.choices[0].text
    text = text.replace("\n","")
    text = text.lstrip('.:')

    return(text)

### 4.2 Query Input 

In [11]:
prompt_widget = widgets.Text(placeholder = 'Write any query as a command', layout=widgets.Layout(height='50px', width='950px'))
display(prompt_widget)


Text(value='', layout=Layout(height='50px', width='950px'), placeholder='Write any query as a command')

Example 1: ``I want to find all employees older than 30 and that also live in Spain``

Example 2: ``Show me all employees that live in the UK or the USA and are a Data Scientist``

### 4.3 Display of the query results

In [12]:
answer = chatgpt_query(prompt_widget.value, header)
filtered_df = eval(answer)
filtered_df

,ID,Name,Occupation,Country,Age
28,29,Madison,Developer,Spain,31
47,48,Bella,Executive,Spain,34
258,259,Everly,Intern,Spain,39
274,275,Sophia,Intern,Spain,35
323,324,Autumn,Accountant,Spain,34
433,434,Harper,Intern,Spain,34
436,437,Grace,Executive,Spain,38
472,473,Delilah,Software Engineer,Spain,38
481,482,Arianna,Data Analyst,Spain,31
492,493,Madeline,Developer,Spain,38


## 5. Google Slides

### 5.1  Defining the permissions

In [13]:
# Set the path to your service account credentials JSON file
service_account_file = 'roche2023.json'

source_slide_id = '190x1G-7DH6zaWEJjTI49sdoJ9ZVr_tyxW05QB_2W-SY' # ID of the Google Slides template

# Necessary authorizations to access the Google Slides template file, create copies and share them
scopes = ['https://www.googleapis.com/auth/drive', 
          'https://www.googleapis.com/auth/drive.file', 
          'https://www.googleapis.com/auth/presentations']

### 5.2 Email input

In [14]:
email_widget = widgets.Text(placeholder = 'Write any query as a command',description = 'Email')
display(email_widget)

Text(value='', description='Email', placeholder='Write any query as a command')

In [16]:
# Build the credentials and Sheets service
credentials = service_account.Credentials.from_service_account_file(service_account_file, scopes=scopes)
drive_service = build('drive', 'v3', credentials=credentials) # Google Drive service
slides_service = build('slides', 'v1', credentials=credentials) # Google Slides service

### 5.3 Creating and sharing Google Slides 

Creates copies of the Google Slides template file and replaces the ``Placeholders`` with the data from the result of the employees query. Shares it with an email address. Informs user upon completion.

Args
* dataframe (df) The filtered df, resulting from the previous query
* share_email (str) An email to share the Google Slides with

In [17]:
def share_slide_copies(dataframe, share_email):
    
    # Defining the Placeholders on the Google Slides
    placeholders_list = ["**Employee ID**", 
                         "**Employee Name**", 
                         "**Occupation**", 
                         "**Country**", 
                         "**Age**"]
    
    # Empty list to store the copies IDs
    copies=[]
    
    # Iterates over the filtered df results 
    for i in range(len(dataframe)):
        
        # Extracts info of the employee from the filtered df
        name = dataframe.iloc[i]['Name']
        employer_id = str(dataframe.iloc[i]['ID'])
        
        # Sets the name of the copied file to be saved with the employee Name and ID
        copy_title = name + ' - ' + employer_id
        copy_body = {'name': copy_title}
        
        # Makes a copy of the template file and stores the response
        copy_response = drive_service.files().copy(fileId=source_slide_id, body=copy_body).execute()
        copies.append(copy_response['id'])
        
        # Store the batch updates requests. There is actually just one request, but a list is expected
        requests = []

        list_of_info = dataframe.iloc[i].tolist()
        list_of_info = [str(item) for item in list_of_info]
        
        # Iterates over the placeholders and the employee info                 
        for placeholder, info in zip(placeholders_list, list_of_info):
            
            # Replaces placeholders info with employee info
            requests.append({"replaceAllText": {"containsText": {"text": placeholder,"matchCase": False},"replaceText": info}})
            slides_service.presentations().batchUpdate(presentationId = copies[i], body={'requests': requests}).execute()

        # Shares the Google Slides copies with the email address
        drive_service.permissions().create(
                fileId=copies[i],
                body={'type': 'user', 'role': 'writer', 'emailAddress': share_email},
                fields = 'id',
                sendNotificationEmail = False
                ).execute()

    print(f"{len(dataframe)} employee files created and shared with {share_email}.")

In [18]:
share_slide_copies(filtered_df, email_widget.value)

20 employee files created and shared with sergi.cmolina@gmail.com.


## 6. Problem solving

By far the biggest issue I encountered was an API error from the Google Drive API informing me about having exceeded sharing quota limits. This error was very missleading because I would get it only at certain times of the day. I did some research on the API documentation where I could find that the recommended course of action was to deploy an [exponential backoff algorithm](https://developers.google.com/drive/api/guides/limits#exponential). 

While on the middle of deploying this, I decided to have a closer look at my current API requests quota and compare it with the usage limits dictated on the API documentation as ``20000 per 100 seconds``. My current quota usages where far lower than the API [usage limit](https://developers.google.com/analytics/devguides/reporting/mcf/v3/limits-quotas#general_quota_limits) as shown in the graph below.


<center>
<img src="https://i.imgur.com/2cw2qkM.png" alt="Image" style="max-width: 600px;"/>
</center>


This error was only being shown when executing the ``share_slide_copies`` function, which made me think the issue had something to do with the file sharing. I tested creating the Google Slides files without sharing them and no error showed. So this led me away from my first assumption. Diving deeper into the matter and also researching other sources like StackOverflow, I was finally able to try and implement a solution that solved the issue. 

Apparently there is a file sharing limit of ``50 files per day`` when sending email notifications which is enabled by default

> drive_service.permissions().create(

>               fileId=copies[i],
>               body={'type': 'user', 'role': 'writer', 'emailAddress': share_email},
>               fields = 'id',
>               sendNotificationEmail = False
>               ).execute()
                

After setting the ``sendNotificationEmail`` parameter to False, everything worked fine.

## 7 Future Improvements 

- Implement Google Speech to pass the query information
- Obtain stronger results by passing also the information contained in the df to have the AI automate even more
- Be able to modify the Google Sheets file with voice commands
- Take into account API usage limit in the functions, unlilekly to exceed with this project
- Pass some of the Google Slides or the Google Sheets features as arguments to the functions so it can be more customizable
- Implement a better Exception handling
- Better User Interface Design on the Desktop App

Sergi Molina